# onc library tutorial


The _onc_ library is developed based on the _requests_ library, which is a popular library for making HTTP requests in Python. In fact, you can use _requests_ alone to interact with the Oceans 3.0 API. But there are cases when you will find the _onc_ library very handy to use (boolean parse, one-click data product download, ability to automatically download all pages, etc.). The tutorial will demonstrate both versions.

This is a Jupyter notebook. You can download the file [here](https://github.com/OceanNetworksCanada/api-python-client/blob/main/doc/source/Tutorial/onc_Library_Tutorial.ipynb).


In [ ]:
# Install some libraries

# 1. onc: this is an onc library tutorial, right?
# 2. request: an alternative (vanilla) way to make HTTP requests to Oceans 3.0 API.
# 3. pandas: because it's useful and fun!
# 4. python-dotenv: a handy library to hide the token outside the notebook.

import sys

!{sys.executable} -m pip install --upgrade requests pandas python-dotenv onc

In [ ]:
# Get the token from your Oceans 3.0 profile page
# Put "TOKEN=[YOUR_TOKEN]" in a .env file.

from dotenv import load_dotenv
import os

load_dotenv(override=True)
token = os.getenv(
    "TOKEN", ""
)  # You can also replace the empty string (second argument) with your token.

In [ ]:
import requests
import pandas as pd
from onc import ONC

onc = ONC(token)

# For not overflowing the max-width of sphinx-rtd-theme
pd.set_option("display.max_colwidth", 30)
pd.set_option("display.max_columns", 5)
pd.set_option("display.max_rows", 5)

## 1. Searching with discovery methods

To download ONC data, you need to specify the type of data you require and where in particular (i.e. location, device) it originates from.

In the Oceans 3.0 API, there's a unique code that identifies every location, device, property, data product type, etc. You include these codes in a group of filters that determine the data you're interested in.

Discovery methods allow you to explore the hierarchy of the ONC database to obtain the codes for your filters (they work like a "search" function).

The example below uses the _getLocations_ method to search for locations that include _"Bullseye"_ in their name (i.e. _"Clayoquot Slope Bullseye Vent"_). It prints a list with a location that matches the search filters provided.


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationName": "Bullseye",
}

# 2. Call methods in the onc library
onc.getLocations(params)

In [ ]:
##### requests #####

# 1. Define your filter parameter
params_requests = {
    "locationName": "Bullseye",
    "token": token,
}

# 2. Define your base url for this query
url = "http://data.oceannetworks.ca/api/locations"

# 3. Run your request
r = requests.get(url, params=params_requests)

# 4. Parse the json file
r.json()

You can see how many deployments the location Bullseye at Clayoquot Slope has until the present date. The column _locationCode_ contains the string "NC89", which is needed for the next steps.


### What _device categories_ are available here at NC89?


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "NC89",
}

# 2. Call methods in the onc library
result = onc.getDeviceCategories(params)

# 3. Read it into a DataFrame
pd.DataFrame(result)

In [ ]:
##### requests #####

# 1. Define your filter parameter
params_requests = {
    "locationCode": "NC89",
    "token": token,
}

# 2. Define your base url for this query
url = "http://data.oceannetworks.ca/api/deviceCategories"

# 3. Run your request
r = requests.get(url, params=params_requests)

# 4. Read it into a DataFrame
pd.DataFrame(r.json())

### What _properties_ are available for the _CTD category_ at this location (NC89)?


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
}

# 2. Call methods in the onc library
r = onc.getProperties(params)

# 3. Read it into a DataFrame
pd.DataFrame(r)

In [ ]:
##### requests #####

# 1. Define your filter parameter
params_requests = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "token": token,
}

# 2. Define your base url for this query
url = "http://data.oceannetworks.ca/api/properties"

# 3. Run your request
r = requests.get(url, params=params_requests)

# 4. Read it into a DataFrame
pd.DataFrame(r.json())

### What _data product types_ are available for the CTD category at this location?


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
}

# 2. Call methods in the onc library
r = onc.getDataProducts(params)

# 3. Read it into a DataFrame
pd.DataFrame(r)

In [ ]:
##### requests #####

# 1. Define your filter parameter
params_requests = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "token": token,
}

# 2. Define your base url for this query
url = "http://data.oceannetworks.ca/api/dataProducts"

# 3. Run your request
r = requests.get(url, params=params_requests)

# 4. Read it into a DataFrame
pd.DataFrame(r.json())

## 2. Downloading data products

Other than using Oceans 3.0 [Data Search](https://data.oceannetworks.ca/DataSearch), we can request the ONC server to generate a **data product**. This is done through the data product download methods.

::: {Note}
This will require three steps before you will be able to see the downloaded data product on your computer:

1. _Request_ the data.
2. _Run_ the Request.
3. _Download_ the Data.

:::

The following example downloads two PNG files with plots for 30 minutes of data from a CTD (find them in the **"output"** folder beside this jupyter notebook). The filter includes codes for **location**, **deviceCategory**, and **dataProduct**, as well as the file **extension** and a time interval. They also include a couple of filters to configure this specific data product type (starting with the **"dpo\_"** prefix) which can be obtained from the [Data Product Options documentation](https://wiki.oceannetworks.ca/display/O2A/Available+Data+Products). You can download more than 120 different types of data products including audio & video.


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dataProductCode": "TSSP",
    "extension": "png",
    "dateFrom": "2017-01-19T00:00:00.000Z",
    "dateTo": "2017-01-19T00:30:00.000Z",
    "dpo_qualityControl": "1",
    "dpo_resample": "none",
}

# 2. Call methods in the onc library
onc.orderDataProduct(params)

In [ ]:
##### requests #####

# 1. Request the data

# Define your base url for this query
url_request = "https://data.oceannetworks.ca/api/dataProductDelivery/request"

# Define your filter parameter
params_requests = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dataProductCode": "TSSP",
    "extension": "png",
    "dateFrom": "2017-01-19T00:00:00.000Z",
    "dateTo": "2017-01-19T00:30:00.000Z",
    "dpo_qualityControl": "1",
    "dpo_resample": "none",
    "token": token,
}

request = requests.get(url_request, params=params_requests)
request.json()

In [ ]:
##### requests continued #####

# 2. Run the request

# Note: you have to execute this cell multiple times until the return shows "status": "complete"
# Note: Depending on your request, you can have more than one file ('fileCount').
#       You will need to individually download these files by using the index parameter.

url_run = "https://data.oceannetworks.ca/api/dataProductDelivery/run"

requestID = request.json()["dpRequestId"]

params_requests = {
    "dpRequestId": requestID,
    "token": token,
}

r = requests.get(url_run, params_requests)
r.json()

In [ ]:
##### requests continued #####

# Find the RunID for the next step
RunId = r.json()[0]["dpRunId"]
RunId

In [ ]:
##### requests continued #####

# 3. Download the data

url_download = "https://data.oceannetworks.ca/api/dataProductDelivery/download"

params_requests = {
    "dpRunId": RunId,
    "token": token,
    "index": "1",
}

r = requests.get(url_download, params_requests)
r  # Rerun this cell until the response code is 200.

# r.headers["Content-Disposition"] has the format "attachement; filename=XXX.png"
# with open(r.headers["Content-Disposition"][22:], 'wb') as f:
#     f.write(r.content)

::: {admonition} Another option to get the data
:class: tip

Obtain your downloads from your user FTP directory (More -> User Directory) in Oceans 3.0.
Navigate to the folder that contains the runId: You will see the files in this folder.

![UserDirectory.png](../_static/Tutorial/onc_Library_Tutorial/UserDirectory.png)

:::


## 3. Obtaining sensor readings in (near) real-time

Once you determine the exact filters that identify the data you are interested in, there are 3 different methods available to download it.

One method allows you to **directly download time-series** of scalar data sensor readings for a given timeframe.

In the following example, we obtain 5 seconds of conductivity readings from the CTD.


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dateFrom": "2020-06-20T00:00:00.000Z",
    "propertyCode": "pressure",
    "dateTo": "2020-06-20T00:00:10.000Z",
}

# 2. Call methods in the onc library
r = onc.getDirectByLocation(params)

# 3. Read it into a DataFrame
pressure = pd.DataFrame(r["sensorData"][0]["data"])
pressure

In [ ]:
##### requests #####

# 1. Define your filter parameter to obtain scalar data for 10 seconds
params_requests = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dateFrom": "2020-06-20T00:00:00.000Z",
    "propertyCode": "pressure",
    "dateTo": "2020-06-20T00:00:10.000Z",
    "token": token,
}

# 2. Define your base url for this query
url = "https://data.oceannetworks.ca/api/scalardata/location"

# 3. Run your request
r = requests.get(url, params_requests)

# 4. Read it into a DataFrame
pressure = pd.DataFrame(r.json()["sensorData"][0]["data"])
pressure

The result includes matching lists of "values" and "sampleTimes". We also use the **property code** "pressure" to limit results to a specific property available in this CTD.

::: {Admonition} Quiz
:class: tip

Where did we get this property from? (Hint: Search "properties" on this page)

:::

Note that we can also get the **raw readings directly** from the device, using the method
**getDirectRawByLocation()**


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dateFrom": "2020-06-20T00:00:00.000Z",
    "dateTo": "2020-06-20T00:00:10.000Z",
}

# 2. Call methods in the onc library
r = onc.getDirectRawByLocation(params)

# 3. Read it into a DataFrame
pd.DataFrame(r["data"])

In [ ]:
##### requests #####

# 1. Define your filter parameter
params_requests = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dateFrom": "2020-06-20T00:00:00.000Z",
    "dateTo": "2020-06-20T00:00:10.000Z",
    "token": token,
}

# 2. Define your base url for this query
url = "http://data.oceannetworks.ca/api/rawdata/location"

# 3. Run your request
r = requests.get(url, params_requests)

# 4. Read it into a DataFrame
pd.DataFrame(r.json()["data"])

## 4. Downloading more data

::: {Admonition} Pagination of response due to too many data rows
:class: note

If the row of the data is above 100,000, not all the data will be returned. The rest of the data can be queried based on the _next_ key in the response.

1. If you use _onc_.

`getDirectRawByLocation` supports a boolean `allPages` parameter. When set to `True`, it will try to retrieve all the pages.

2. If you use _requests_.

You have to manually query the next pages until the `next` key in the response json is `None`, and concatenate all the data together.

:::


In [ ]:
##### onc #####

# 1. Define your filter parameter with a longer date range
params_longer_range = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dateFrom": "2020-06-20T00:00:00.000Z",
    "dateTo": "2020-06-21T20:00:00.000Z",
}

# 2. Call methods in the onc library
r = onc.getDirectRawByLocation(params_longer_range, allPages=True)

# 3. Read it into a DataFrame
pd.DataFrame(r["data"])

In [ ]:
##### requests #####

# 1. Define your filter parameter with a longer date range
params_requests_longer_range = {
    "locationCode": "NC89",
    "deviceCategoryCode": "CTD",
    "dateFrom": "2020-06-20T00:00:00.000Z",
    "dateTo": "2020-06-21T20:00:00.000Z",
    "token": token,
}

# 2. Define your base url for this query
url = "http://data.oceannetworks.ca/api/rawdata/location"

# 3. Run your request (the url is still the same)
r = requests.get(url, params_requests_longer_range)

# 4. Read it into a DataFrame
pd.DataFrame(r.json()["data"])

In [ ]:
r.json()["next"]

In [ ]:
# Update the dateFrom parameter to get the next page
params_requests_longer_range["dateFrom"] = r.json()["next"]["parameters"]["dateFrom"]
r = requests.get(url, params_requests_longer_range)
pd.DataFrame(r.json()["data"])

In [ ]:
print(r.json()["next"])

## 5. Downloading archived files

A faster way to download data products (if it suits your needs) is to leverage how ONC scripts auto-generate and archive data products of different types at set time intervals. You can directly download these data product files from our files archive, as long as you know their unique filename.

In the following example, we get the list of archived files available for a camera at Ridley Island (in a certain time range).


In [ ]:
##### onc #####

# 1. Define your filter parameter
params = {
    "locationCode": "RISS",
    "deviceCategoryCode": "VIDEOCAM",
    "dateFrom": "2016-12-01T00:00:00.000Z",
    "dateTo": "2016-12-01T00:05:00.000Z",
}

# 2. Call methods in the onc library
r = onc.getListByLocation(params)
r["files"]

In [ ]:
##### requests #####

# 1. Define your filter parameter
params_requests = {
    "locationCode": "RISS",
    "deviceCategoryCode": "VIDEOCAM",
    "dateFrom": "2016-12-01T00:00:00.000Z",
    "dateTo": "2016-12-01T00:05:00.000Z",
    "token": token,
}

# 2. Define your base url for this query
url_location = "https://data.oceannetworks.ca/api/archivefile/location"

# 3. Run your request
r = requests.get(url_location, params_requests)
r.json()["files"]

Once we have the file name, you can use the method **"getFile()"** to download individual files:


In [ ]:
##### onc #####

# 1. Call methods in the onc library with the filename. The file is downloaded in the output folder.
onc.getFile("AXISQ6044PTZACCC8E334C53_20161201T000001.000Z.jpg")

In [ ]:
##### requests #####

# 1. Define your filter parameter with the filename
params = {
    "filename": "AXISQ6044PTZACCC8E334C53_20161201T000001.000Z.jpg",
    "token": token,
}

# 2. Define your base url for this query
url_location = "https://data.oceannetworks.ca/api/archivefile/download"

# 3. Run your request (the url is still the same)
r = requests.get(url_location, params)

# 4. Save the file
# with open(params['filename'], 'wb') as f:
#     f.write(r.content)